In [4]:
import argparse
import qlib
import ruamel.yaml as yaml
from qlib.utils import init_instance_by_config
import pandas as pd
import torch
from qlib.contrib.evaluate import (
    backtest as normal_backtest,
    risk_analysis,
)

In [5]:
parser = argparse.ArgumentParser(allow_abbrev=False)
parser.add_argument("--seed", type=int, default=1000, help="random seed")
parser.add_argument("--config_file", type=str, default="us_latest.yaml", help="config file")
parser.add_argument("-f")
args = parser.parse_args()


seed = args.seed
config_file = args.config_file

# set random seed
with open(config_file) as f:
    config = yaml.safe_load(f)

# seed_suffix = "/seed1000" if "init" in config_file else f"/seed{seed}"
seed_suffix = ""
config["task"]["model"]["kwargs"].update(
    {"seed": seed, "logdir": config["task"]["model"]["kwargs"]["logdir"] + seed_suffix}
)

# initialize workflow
qlib.init(
    provider_uri=config["qlib_init"]["provider_uri"],
    region=config["qlib_init"]["region"],
)

[3413:MainThread](2021-10-30 13:21:25,158) INFO - qlib.Initialization - [config.py:386] - default_conf: client.
[3413:MainThread](2021-10-30 13:21:25,165) WARNING - qlib.Initialization - [config.py:411] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[3413:MainThread](2021-10-30 13:21:25,171) INFO - qlib.Initialization - [__init__.py:56] - qlib successfully initialized based on client settings.
[3413:MainThread](2021-10-30 13:21:25,173) INFO - qlib.Initialization - [__init__.py:58] - data_path={'__DEFAULT_FREQ': PosixPath('/home/us_data_dump')}


In [6]:
latest_dataset = init_instance_by_config(config["task"]["dataset"])

[3413:MainThread](2021-10-30 13:23:07,420) INFO - qlib.timer - [log.py:113] - Time cost: 99.618s | Loading data Done
[3413:MainThread](2021-10-30 13:23:07,436) INFO - qlib.timer - [log.py:113] - Time cost: 0.010s | FilterCol Done
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:7349: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._clip_with_scalar(lower, upper, inplace=inplace)
[3413:MainThread](2021-10-30 13

In [7]:
model = init_instance_by_config(config["task"]["model"])

Please install necessary libs for CatBoostModel.
Please install necessary libs for XGBModel, such as xgboost.


[3413:MainThread](2021-10-30 13:23:07,939) WARNING - qlib.TRA - [pytorch_tra.py:100] - `eval_train` will be ignored when using TRA.router
[3413:MainThread](2021-10-30 13:23:07,943) WARNING - qlib.TRA - [pytorch_tra.py:132] - logdir output/nasdaq is not empty
[3413:MainThread](2021-10-30 13:23:07,944) INFO - qlib.TRA - [pytorch_tra.py:141] - init TRAModel...
[3413:MainThread](2021-10-30 13:23:07,955) WARNING - qlib.TRA - [pytorch_tra.py:150] - load state dict from `init_state`
[3413:MainThread](2021-10-30 13:23:07,962) WARNING - qlib.TRA - [pytorch_tra.py:154] - {'unexpected_keys': [], 'missing_keys': [], 'error_msgs': []}
[3413:MainThread](2021-10-30 13:23:07,963) INFO - qlib.TRA - [pytorch_tra.py:171] - # model params: 59520
[3413:MainThread](2021-10-30 13:23:07,964) INFO - qlib.TRA - [pytorch_tra.py:172] - # tra params: 5606


RNN(
  (rnn): LSTM(20, 64, num_layers=2, batch_first=True)
  (W): Linear(in_features=64, out_features=64, bias=True)
  (u): Linear(in_features=64, out_features=1, bias=False)
)
TRA(
  (predictors): Linear(in_features=128, out_features=3, bias=True)
  (router): LSTM(3, 32, batch_first=True)
  (fc): Linear(in_features=160, out_features=3, bias=True)
)


In [8]:
state_dict = torch.load("/home/qlib_test/examples/benchmarks/TRA/output/us_158_nassp_epoch100/model.bin", map_location="cpu")["model"]

In [9]:
model.model.load_state_dict(state_dict)

<All keys matched successfully>

In [10]:
model.fitted = True

In [11]:
pred = model.predict(latest_dataset)

100%|████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  7.48it/s]
[3413:MainThread](2021-10-30 13:23:10,699) INFO - qlib.TRA - [pytorch_tra.py:516] - test metrics: {'MSE': 0.17033045541617953, 'MAE': 0.3362955980771043, 'IC': -0.021986720446414757, 'ICIR': -0.21481076826114678}


In [15]:
reset_df = pred.reset_index('instrument')

In [21]:
import qlib
from qlib.data import D
qlib.init(provider_uri="/home/us_data_dump", region="us")

df = D.features(D.instruments("nassp"), ["$open", "$factor"], freq="day")

past_data = df.reset_index('instrument')

[3413:MainThread](2021-10-30 14:41:46,939) INFO - qlib.Initialization - [config.py:386] - default_conf: client.
[3413:MainThread](2021-10-30 14:41:46,942) WARNING - qlib.Initialization - [config.py:411] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[3413:MainThread](2021-10-30 14:41:46,944) INFO - qlib.Initialization - [__init__.py:56] - qlib successfully initialized based on client settings.
[3413:MainThread](2021-10-30 14:41:46,946) INFO - qlib.Initialization - [__init__.py:58] - data_path={'__DEFAULT_FREQ': PosixPath('/home/us_data_dump')}


In [40]:
import time
import numpy as np

def backtest(top_k, max_buying, fee, p=False, early_stop=True, money=1000.0):
    st = time.time()
    indexs = reset_df.index.drop_duplicates()
    cnt = 0 
    # money = 100
    current_buying = {}
    num_buy = 0
    loss_cut = 0.95
    my_stack = None

    for idx in indexs:
        cnt+=1
        today_buy_list = []
        today_sell_list = []
        today_earn = 0
        
        try:
            if p:
                print("---------------------------------")
                print("day ", cnt, idx)
                print("---------------------------------")
                
            ## predict today score
            if type(reset_df.loc[idx]['score']) is np.float32:
                stack = reset_df.loc[idx]
            else:
                stack = reset_df.loc[idx].sort_values(by='score' , ascending=True)
                # stack = stack[stack['score'] < -0.17]
            
            ## update current stocks' prices, money
            # print("money, can_buy_money:", money, can_buy_money)
            can_buy_money = money
            current_buying_list = list(current_buying.keys())
            for cur_stock in current_buying_list:
                today_stock_price = df.loc[cur_stock,idx][0] / df.loc[cur_stock,idx][1]
                today_stock_price = today_stock_price.round(2)
                current_buying[cur_stock][2] = today_stock_price.round(2)
                current_buying[cur_stock][4] = (current_buying[cur_stock][3] * today_stock_price).round(2)
                ### update current total money
                if not np.isnan(current_buying[cur_stock][4]):
                    can_buy_money += current_buying[cur_stock][4].round(2)
            # print("money, can_buy_money:", money, can_buy_money)
            
            if p:
                print("current stocks: ", current_buying)
                
            ## sorting current stocks' score
            my_stack = stack[stack['instrument'].isin(current_buying_list)]
            my_stack = my_stack.sort_values(by='score' , ascending=True)
            my_stack_list = list(my_stack['instrument'])
            
            # print("current stocks' score:", my_stack_list)
            
            ## select buy_list and sell_list
            today_buy_cnt = 0
            for x in stack['instrument']:
                if today_buy_cnt >= top_k:
                    break
                cur_stock = x
                
                if cur_stock in my_stack_list:
                    # today_buy_list.append(cur_stock)
                    # today_sell_list.append(cur_stock)
                    today_buy_cnt+=1
                    if not np.isnan(current_buying[cur_stock][4]):
                        can_buy_money -= current_buying[cur_stock][4].round(2)
                    continue
                
                ### assert we can buy this stock
                today_stock_price = df.loc[cur_stock,idx][0] / df.loc[cur_stock,idx][1]
                today_stock_price = today_stock_price.round(2)
                if today_stock_price > can_buy_money / (max_buying - today_buy_cnt):
                    continue
                ### assert price is not nan
                if np.isnan(today_stock_price):
                    continue
                
                ### sell
                if num_buy >= max_buying:
                    last_stock = my_stack_list[-1]
                    num_last_stock = current_buying[last_stock][0]

                    num_buy -= 1
                    today_sell_list.append(last_stock)
                    if num_last_stock == 1:
                        earn = current_buying[last_stock][4]
                        del current_buying[last_stock]
                        my_stack_list = my_stack_list[:-1]
                    else:
                        current_buying[last_stock][0] -= 1
                        current_buying[last_stock][3] -= sell_total
                        current_buying[last_stock][4] -= sell_total*end
                        earn = sell_total*end
                    if not np.isnan(earn):
                        money += earn.round(2)*(1.0-fee)

                ### buy
                total = 1
                while(1):
                    total_price = (total+1)*today_stock_price
                    if total_price < can_buy_money / (max_buying - today_buy_cnt):
                        total+=1
                    else:
                        break
                total_price = total*today_stock_price
                total_price = total_price.round(2)
                
                num_buy+=1
                today_buy_cnt += 1
                today_buy_list.append(cur_stock)
                if cur_stock not in current_buying:
                    current_buying[x] = [1, today_stock_price, today_stock_price, total, total_price] #num, start_price, current_price, total, total_price
                else:
                    current_buying[x][0] += 1
                    current_buying[x][3] += total
                    current_buying[x][4] += total_price
                if not np.isnan(total_price):
                    money -= total_price.round(2)*(1.0+fee)
                    can_buy_money -= total_price.round(2)*(1.0+fee)
            if p:
                print("buy: ", today_buy_list)
                print("sell: ", today_sell_list)
            # print("current stocks: ", current_buying)
            
            ## caluate current_money
            current_money = money
            for c in current_buying:
                total_price = current_buying[c][4]
                current_money += total_price
            if money<0:
                print("?")
            if p:
                print("current stocks: ", len(current_buying), current_buying)
                print("current usable money: ", money)
                print("current money: ", current_money)
        except Exception as e:
            print(e)
            if early_stop and cnt >= 800:
                print("---------------------------------")
                print("config: ", top_k, max_buying)
                print("day ", cnt, idx)
                print("current stocks: ", len(current_buying), current_buying)
                print("current money: ", current_money)
                print("time:", time.time() - st)
                break
            continue
        if early_stop and cnt >= 800:
            print("---------------------------------")
            print("config: ", top_k, max_buying)
            print("day ", cnt, idx)
            print("current stocks: ", len(current_buying), current_buying)
            print("current usable money: ", money)
            print("current money: ", current_money)
            print("time:", time.time() - st)
            break
    return current_money

In [42]:
backtest(1, 1, 0.0007, money=10000, p=True)

---------------------------------
day  1 2021-10-01 00:00:00
---------------------------------
current stocks:  {}
buy:  ['PAYC']
sell:  []
current stocks:  1 {'PAYC': [1, 497.53, 497.53, 20, 9950.6]}
current usable money:  42.434580000001006
current money:  9993.034580000001
---------------------------------
day  2 2021-10-04 00:00:00
---------------------------------
current stocks:  {'PAYC': [1, 497.53, 500.0, 20, 10000.0]}
buy:  []
sell:  []
current stocks:  1 {'PAYC': [1, 497.53, 500.0, 20, 10000.0]}
current usable money:  42.434580000001006
current money:  10042.434580000001
---------------------------------
day  3 2021-10-05 00:00:00
---------------------------------
current stocks:  {'PAYC': [1, 497.53, 495.75, 20, 9915.0]}
buy:  ['SLB']
sell:  ['PAYC']
current stocks:  1 {'SLB': [1, 31.26, 31.26, 318, 9940.68]}
current usable money:  2.8556040000003122
current money:  9943.535604
---------------------------------
day  4 2021-10-06 00:00:00
---------------------------------
cur

9435.744468000006

In [63]:
import time
import numpy as np
def backtest_sell_all(top_k, max_buying, fee, p=False, early_stop=True):
    st = time.time()
    indexs = reset_df.index.drop_duplicates()
    cnt = 0 
    money = 100
    current_buying = {}
    num_buy = 0
    loss_cut = 0.95
    my_stack = None
    
    for idx in indexs:
        today_buy_list = []
        today_sell_list = []
        
        cnt+=1
        try:
            if p:
                print("---------------------------------")
                print("day ", cnt, idx)
                print("---------------------------------")
            if type(reset_df.loc[idx]['score']) is np.float32:
                stack = reset_df.loc[idx]
            else:
                stack = reset_df.loc[idx].sort_values(by='score' , ascending=True)

            # print(stack)
                
            current_buying_list = list(current_buying.keys())
            for cur_stock in current_buying_list:
                today_stock_price = df.loc[cur_stock,idx][0].round(8)
                current_buying[cur_stock][2] = today_stock_price

            my_stack = stack[stack['instrument'].isin(current_buying_list)]
            my_stack = my_stack.sort_values(by='score' , ascending=True)
            my_stack_list = list(my_stack['instrument'])
            for x in stack['instrument'][:top_k]:
                if num_buy >= max_buying:
                    last_stock = my_stack_list[-1]
                    num_last_stock = current_buying[last_stock][0]

                    num_buy -= 1
                    # start, end = current_buying[last_stock][1:]
                    # if not (np.isnan(start) or np.isnan(end)):
                    #     earn = (end - start) / start * money / max_buying
                    #     earn -= 2 * fee * money / max_buying
                    today_sell_list.append(last_stock)
                    # if p:
                    #     print("sell: ", last_stock, current_buying[last_stock])
                    #     money += earn

                    if num_last_stock == 1:
                        del current_buying[last_stock]
                        my_stack_list = my_stack_list[:-1]
                    else:
                        current_buying[last_stock][0] -= 1

                if x not in current_buying:
                    cur_stock = x
                    today_stock_price = df.loc[cur_stock,idx][0].round(6)
                    current_buying[x] = [1, today_stock_price, today_stock_price] #num, start_price, current_price
                else:
                    current_buying[x][0] += 1
                num_buy +=1
                today_buy_list.append(x)
                # if p:
                #     print("buy: ", x, current_buying[x])
            
            # print(my_stack_list)
            # print(stack['instrument'][:top_k])
            # print("buy: ", today_buy_list)
            # print("sell: ", today_sell_list)
            for idx1, b in enumerate(today_buy_list):
                for idx2, s in enumerate(today_sell_list):
                    if b==s:
                        today_buy_list[idx1] = ''
                        today_sell_list[idx2] = ''
                        break
            today_buy_list = list(filter(lambda a: a!='', today_buy_list))
            today_sell_list = list(filter(lambda a: a!='', today_sell_list))
            
            for selled in today_sell_list:
                start, end = current_buying[last_stock][1:]
                if not (np.isnan(start) or np.isnan(end)):
                    earn = (end - start) / start * money / max_buying
                    earn -= 2 * fee * money / max_buying
            
            if p:
                # print("config: ", top_k, max_buying)
                print("current stocks: ", current_buying)
                print("current money: ", money)
                print("time:", time.time() - st)
                print("buy: ", today_buy_list)
                print("sell: ", today_sell_list)
        except Exception as e:
            print(e)
            if early_stop and cnt >= 800:
                print("---------------------------------")
                print("config: ", top_k, max_buying)
                print("day ", cnt, idx)
                print(num_buy, current_buying)
                print("current money: ", money)
                print("time:", time.time() - st)
                break
            continue
        if early_stop and cnt >= 800:
            print("---------------------------------")
            print("config: ", top_k, max_buying)
            print("day ", cnt, idx)
            print(num_buy, current_buying)
            print("current money: ", money)
            print("time:", time.time() - st)
            break
    return money

In [64]:
backtest_sell_all(5, 5, 0.0007, p=True)

---------------------------------
day  1 2021-10-01 00:00:00
---------------------------------
buy:  ['NTAP', 'PAYC', 'MS', 'VMC', 'HFC']
sell:  []
current stocks:  {'NTAP': [1, 1.865642, 1.865642], 'PAYC': [1, 1.304278, 1.304278], 'MS': [1, 1.767435, 1.767435], 'VMC': [1, 1.203536, 1.203536], 'HFC': [1, 1.601305, 1.601305]}
current money:  100
time: 0.008930444717407227
buy:  ['NTAP', 'PAYC', 'MS', 'VMC', 'HFC']
sell:  []
---------------------------------
day  2 2021-10-04 00:00:00
---------------------------------
buy:  ['TSCO', 'NTAP', 'EXC', 'COP', 'RF']
sell:  ['VMC', 'PAYC', 'HFC', 'MS', 'NTAP']
current stocks:  {'NTAP': [1, 1.865642, 1.8797219], 'TSCO': [1, 1.498544, 1.498544], 'EXC': [1, 1.12794, 1.12794], 'COP': [1, 2.05805, 2.05805], 'RF': [1, 1.507089, 1.507089]}
current money:  100
time: 0.019327878952026367
buy:  ['TSCO', 'EXC', 'COP', 'RF']
sell:  ['VMC', 'PAYC', 'HFC', 'MS']
---------------------------------
day  3 2021-10-05 00:00:00
---------------------------------
bu

100

In [46]:
buy =  ['VLO', 'HFC', 'AIG', 'HPQ', 'EMR']
sell =  ['VLO', 'HFC', 'AIG', 'HPQ', 'HPQ']

In [50]:
remove_buy = []
remove_sell = []
for idx1, b in enumerate(buy):
    for idx2, s in enumerate(sell):
        if b==s:
            buy[idx1] = ''
            sell[idx2] = ''
            break
print(buy,sell)
# buy = 

['', '', '', '', 'EMR'] ['', '', '', '', 'HPQ']


In [56]:
buy = list(filter(lambda a: a!='', buy))
sell = list(filter(lambda a: a!='', sell))

In [57]:
print(buy, sell)

['EMR'] ['HPQ']


In [ ]:
b